In [2]:
import spotipy
import random
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

from spotify_config import config

client_credentials_manager = SpotifyClientCredentials(config['client_id'],
                                                      config['client_secret'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [3]:
def get_random_query():
    # A list of all characters that can be chosen.
    characters = 'abcdefghijklmnopqrstuvwxyz'
    # characters = 'αβγδεζηθικλμνξοπρστυφχψως'
    # characters = 'йцукенгшщзфывапролдячсмитьбюэжхъ'

    country_codes = ['AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 
        'SV', 'EE', 'FI', 'FR', 'DE', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'ID', 'IE', 'IT', 'JP', 'LV', 'LI', 'LT', 'LU', 'MY', 'MT', 'MX', 'MC', 'NL', 
        'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'ES', 'SK', 'SE', 'CH', 'TW', 'TR', 'GB', 'US', 'UY']
    
    # Gets a random character from the characters string.
    random_character = random.choice(characters)
    random_search = ''

    random_market = random.choice(country_codes)
    offset = random.randint(0, 500)


    # Places the wildcard character at the beginning, or both beginning and end, randomly.
    rand = random.randint(0, 1)
    if rand == 0:
        random_search = random_character + '%'
    else:
        random_search = '%' + random_character + '%'
    return random_search, random_market, offset


In [4]:
SONGS_LIMIT = 15000
songs = set()

In [5]:
while len(songs) < SONGS_LIMIT:
    random_search = get_random_query()
    random_search, random_market, offset = get_random_query()
    print(f"Searching for {random_search} {offset} in market {random_market}, current songs length {len(songs)}")
    
    result = sp.search(q=random_search, type="track", limit=50, offset=offset, market=random_market)
    items = result["tracks"]["items"]
    current_songs = {(song["name"], song["id"], song["artists"][0]["name"]) for song in items}
    songs = songs.union(current_songs)

Searching for %w% 133 in market DO, current songs length 0
Searching for %g% 488 in market IE, current songs length 50
Searching for w% 258 in market NL, current songs length 99
Searching for x% 125 in market NO, current songs length 146
Searching for c% 402 in market AR, current songs length 196
Searching for %o% 397 in market US, current songs length 246
Searching for w% 459 in market AU, current songs length 296
Searching for %d% 385 in market FI, current songs length 342
Searching for %o% 277 in market CH, current songs length 391
Searching for %f% 429 in market CL, current songs length 437
Searching for o% 316 in market NO, current songs length 486
Searching for %i% 303 in market NL, current songs length 528
Searching for l% 10 in market DO, current songs length 572
Searching for i% 49 in market GR, current songs length 618
Searching for n% 5 in market LU, current songs length 664
Searching for %b% 386 in market AT, current songs length 705
Searching for j% 310 in market MT, curre

In [6]:
songs

{('Una Vez', '5J7teKIJGtqVypgYI6sqXj', 'Bad Bunny'),
 ('¡Corre!', '5kIcrM3QVD4BQPFMszQnU1', 'Jesse & Joy'),
 ('Shining One', '7DTKSwO1kTAJDu9Q5U73UZ', 'BE:FIRST'),
 ('Stop And Stare', '1KYboK62JPwNXnzX9V71G8', 'OneRepublic'),
 ('Jab Koi Baat - Recreated', '43GfQ1SC17rBWiHkOtdwK8', 'Atif Aslam'),
 ('Ya No Somos Ni Seremos', '0UXlu64mDLvfzR8IXMz06J', 'Christian Nodal'),
 ('Kyusi', '4IMI7CdeNMDzq544pAgXqj', 'Zild'),
 ('Dragon Roll', '2fCdtNreNptDAZnisSOLpX', 'Frenna'),
 ('Te Quiero X Eso', '6MXtWx5mWkj8XJaEqouzCW', 'MYA'),
 ('Kusut', '3diD8b3u6oEn74BezQpujj', 'Fourtwnty'),
 ('My old story', '2jDnaEpQI6xjahWZzeVyaM', 'IU'),
 ('TENGO FE', '2qqDYvxtre2yngC0yxKSKg', 'Feid'),
 ('In Love With You', '19uJ0h4T2kG5pQX30k6GSy', 'Regine Velasquez'),
 ('Marea (We’ve Lost Dancing)', '1t0Jmqg1pKVBbxjQFZebeR', 'Fred again..'),
 ('Lean On Me', '5acSb48zFAcXTdL5Wsk8xx', 'Cheat Codes'),
 ('Dancing in the Moonlight', '0q21FNwES2bbtcduB6kjEU', 'King Harvest'),
 ('Pareja Del Año', '7r6Dk2SlpVT4cnc61nNL5J', 'S

In [7]:
len(songs)

15006

In [8]:
df = pd.DataFrame()
df["id"] = [song[1] for song in songs]
df["title"] = [song[0] for song in songs]
df["main_artist"] = [song[2] for song in songs]

In [9]:
features = {}
all_track_ids = list(df['id'])

In [10]:
def get_features(start, num_tracks):
    while start < len(all_track_ids):
        print(f'getting from {start} to {start+num_tracks}')
        tracks_batch = all_track_ids[start:start+num_tracks]
        features_batch = sp.audio_features(tracks_batch)
        features.update({ track_id : track_features 
                        for track_id, track_features in zip(tracks_batch, features_batch) })
        start += num_tracks

In [11]:
get_features(0, 100)

getting from 0 to 100
getting from 100 to 200
getting from 200 to 300
getting from 300 to 400
getting from 400 to 500
getting from 500 to 600
getting from 600 to 700
getting from 700 to 800
getting from 800 to 900
getting from 900 to 1000
getting from 1000 to 1100
getting from 1100 to 1200
getting from 1200 to 1300
getting from 1300 to 1400
getting from 1400 to 1500
getting from 1500 to 1600
getting from 1600 to 1700
getting from 1700 to 1800
getting from 1800 to 1900
getting from 1900 to 2000
getting from 2000 to 2100
getting from 2100 to 2200
getting from 2200 to 2300
getting from 2300 to 2400
getting from 2400 to 2500
getting from 2500 to 2600
getting from 2600 to 2700
getting from 2700 to 2800
getting from 2800 to 2900
getting from 2900 to 3000
getting from 3000 to 3100
getting from 3100 to 3200
getting from 3200 to 3300
getting from 3300 to 3400
getting from 3400 to 3500
getting from 3500 to 3600
getting from 3600 to 3700
getting from 3700 to 3800
getting from 3800 to 3900
getting

In [12]:
tracks = pd.DataFrame.from_dict(features)
tracks = tracks.T
tracks = tracks.reset_index().rename(columns={'index' : 'song_id'})
tracks

,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,5J7teKIJGtqVypgYI6sqXj,0.55,0.598,11,-5.402,0,0.0831,0.132,0.000003,0.0955,0.543,83.836,audio_features,5J7teKIJGtqVypgYI6sqXj,spotify:track:5J7teKIJGtqVypgYI6sqXj,https://api.spotify.com/v1/tracks/5J7teKIJGtqV...,https://api.spotify.com/v1/audio-analysis/5J7t...,232496,4
1,5kIcrM3QVD4BQPFMszQnU1,0.479,0.477,11,-6.886,1,0.0363,0.127,0,0.15,0.127,151.97,audio_features,5kIcrM3QVD4BQPFMszQnU1,spotify:track:5kIcrM3QVD4BQPFMszQnU1,https://api.spotify.com/v1/tracks/5kIcrM3QVD4B...,https://api.spotify.com/v1/audio-analysis/5kIc...,288893,4
2,7DTKSwO1kTAJDu9Q5U73UZ,0.716,0.737,7,-5.671,0,0.0851,0.115,0,0.138,0.573,144.986,audio_features,7DTKSwO1kTAJDu9Q5U73UZ,spotify:track:7DTKSwO1kTAJDu9Q5U73UZ,https://api.spotify.com/v1/tracks/7DTKSwO1kTAJ...,https://api.spotify.com/v1/audio-analysis/7DTK...,244107,4
3,1KYboK62JPwNXnzX9V71G8,0.329,0.866,4,-4.328,1,0.0484,0.0724,0,0.0869,0.257,184.793,audio_features,1KYboK62JPwNXnzX9V71G8,spotify:track:1KYboK62JPwNXnzX9V71G8,https://api.spotify.com/v1/tracks/1KYboK62JPwN...,https://api.spotify.com/v1/audio-analysis/1KYb...,223800,4
4,43GfQ1SC17rBWiHkOtdwK8,0.667,0.621,5,-5.697,1,0.0271,0.47,0.000041,0.13,0.451,106.032,audio_features,43GfQ1SC17rBWiHkOtdwK8,spotify:track:43GfQ1SC17rBWiHkOtdwK8,https://api.spotify.com/v1/tracks/43GfQ1SC17rB...,https://api.spotify.com/v1/audio-analysis/43Gf...,192229,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15001,6YrbjsY01cgcuvNhe2SQ1Z,0.673,0.663,1,-8.282,1,0.33,0.114,0.000016,0.192,0.442,95.466,audio_features,6YrbjsY01cgcuvNhe2SQ1Z,spotify:track:6YrbjsY01cgcuvNhe2SQ1Z,https://api.spotify.com/v1/tracks/6YrbjsY01cgc...,https://api.spotify.com/v1/audio-analysis/6Yrb...,100000,4
15002,6cmn21bodxviaGAHa5ZXbF,0.698,0.581,0,-7.218,1,0.0444,0.262,0.000798,0.165,0.47,149.038,audio_features,6cmn21bodxviaGAHa5ZXbF,spotify:track:6cmn21bodxviaGAHa5ZXbF,https://api.spotify.com/v1/tracks/6cmn21bodxvi...,https://api.spotify.com/v1/audio-analysis/6cmn...,167534,4
15003,6y6xhAgZjvxy5kR5rigpY3,0.594,0.64,10,-6.865,1,0.033,0.0169,0,0.135,0.492,92.871,audio_features,6y6xhAgZjvxy5kR5rigpY3,spotify:track:6y6xhAgZjvxy5kR5rigpY3,https://api.spotify.com/v1/tracks/6y6xhAgZjvxy...,https://api.spotify.com/v1/audio-analysis/6y6x...,170457,4
15004,0PpfKq10NLf5vGFm0CPx3j,0.713,0.794,7,-5.693,1,0.177,0.038,0.00001,0.117,0.304,116.033,audio_features,0PpfKq10NLf5vGFm0CPx3j,spotify:track:0PpfKq10NLf5vGFm0CPx3j,https://api.spotify.com/v1/tracks/0PpfKq10NLf5...,https://api.spotify.com/v1/audio-analysis/0Ppf...,171724,4


In [13]:
tracks = tracks.merge(df, left_on="song_id", right_on="id")


In [14]:
tracks

,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,type,id_x,uri,track_href,analysis_url,duration_ms,time_signature,id_y,title,main_artist
0,5J7teKIJGtqVypgYI6sqXj,0.55,0.598,11,-5.402,0,0.0831,0.132,0.000003,0.0955,...,audio_features,5J7teKIJGtqVypgYI6sqXj,spotify:track:5J7teKIJGtqVypgYI6sqXj,https://api.spotify.com/v1/tracks/5J7teKIJGtqV...,https://api.spotify.com/v1/audio-analysis/5J7t...,232496,4,5J7teKIJGtqVypgYI6sqXj,Una Vez,Bad Bunny
1,5kIcrM3QVD4BQPFMszQnU1,0.479,0.477,11,-6.886,1,0.0363,0.127,0,0.15,...,audio_features,5kIcrM3QVD4BQPFMszQnU1,spotify:track:5kIcrM3QVD4BQPFMszQnU1,https://api.spotify.com/v1/tracks/5kIcrM3QVD4B...,https://api.spotify.com/v1/audio-analysis/5kIc...,288893,4,5kIcrM3QVD4BQPFMszQnU1,¡Corre!,Jesse & Joy
2,7DTKSwO1kTAJDu9Q5U73UZ,0.716,0.737,7,-5.671,0,0.0851,0.115,0,0.138,...,audio_features,7DTKSwO1kTAJDu9Q5U73UZ,spotify:track:7DTKSwO1kTAJDu9Q5U73UZ,https://api.spotify.com/v1/tracks/7DTKSwO1kTAJ...,https://api.spotify.com/v1/audio-analysis/7DTK...,244107,4,7DTKSwO1kTAJDu9Q5U73UZ,Shining One,BE:FIRST
3,1KYboK62JPwNXnzX9V71G8,0.329,0.866,4,-4.328,1,0.0484,0.0724,0,0.0869,...,audio_features,1KYboK62JPwNXnzX9V71G8,spotify:track:1KYboK62JPwNXnzX9V71G8,https://api.spotify.com/v1/tracks/1KYboK62JPwN...,https://api.spotify.com/v1/audio-analysis/1KYb...,223800,4,1KYboK62JPwNXnzX9V71G8,Stop And Stare,OneRepublic
4,43GfQ1SC17rBWiHkOtdwK8,0.667,0.621,5,-5.697,1,0.0271,0.47,0.000041,0.13,...,audio_features,43GfQ1SC17rBWiHkOtdwK8,spotify:track:43GfQ1SC17rBWiHkOtdwK8,https://api.spotify.com/v1/tracks/43GfQ1SC17rB...,https://api.spotify.com/v1/audio-analysis/43Gf...,192229,4,43GfQ1SC17rBWiHkOtdwK8,Jab Koi Baat - Recreated,Atif Aslam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15001,6YrbjsY01cgcuvNhe2SQ1Z,0.673,0.663,1,-8.282,1,0.33,0.114,0.000016,0.192,...,audio_features,6YrbjsY01cgcuvNhe2SQ1Z,spotify:track:6YrbjsY01cgcuvNhe2SQ1Z,https://api.spotify.com/v1/tracks/6YrbjsY01cgc...,https://api.spotify.com/v1/audio-analysis/6Yrb...,100000,4,6YrbjsY01cgcuvNhe2SQ1Z,Baccarat,Kalazh44
15002,6cmn21bodxviaGAHa5ZXbF,0.698,0.581,0,-7.218,1,0.0444,0.262,0.000798,0.165,...,audio_features,6cmn21bodxviaGAHa5ZXbF,spotify:track:6cmn21bodxviaGAHa5ZXbF,https://api.spotify.com/v1/tracks/6cmn21bodxvi...,https://api.spotify.com/v1/audio-analysis/6cmn...,167534,4,6cmn21bodxviaGAHa5ZXbF,Uber Driver,mishlawi
15003,6y6xhAgZjvxy5kR5rigpY3,0.594,0.64,10,-6.865,1,0.033,0.0169,0,0.135,...,audio_features,6y6xhAgZjvxy5kR5rigpY3,spotify:track:6y6xhAgZjvxy5kR5rigpY3,https://api.spotify.com/v1/tracks/6y6xhAgZjvxy...,https://api.spotify.com/v1/audio-analysis/6y6x...,170457,4,6y6xhAgZjvxy5kR5rigpY3,Chasing Stars (feat. James Bay),Alesso
15004,0PpfKq10NLf5vGFm0CPx3j,0.713,0.794,7,-5.693,1,0.177,0.038,0.00001,0.117,...,audio_features,0PpfKq10NLf5vGFm0CPx3j,spotify:track:0PpfKq10NLf5vGFm0CPx3j,https://api.spotify.com/v1/tracks/0PpfKq10NLf5...,https://api.spotify.com/v1/audio-analysis/0Ppf...,171724,4,0PpfKq10NLf5vGFm0CPx3j,Kids on Whizz,Alok


In [15]:
tracks.to_csv("songs15000.csv", index=False)

In [16]:
features = {}
songs_subset = tracks.head(500)
songs_subset

,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,type,id_x,uri,track_href,analysis_url,duration_ms,time_signature,id_y,title,main_artist
0,5J7teKIJGtqVypgYI6sqXj,0.55,0.598,11,-5.402,0,0.0831,0.132,0.000003,0.0955,...,audio_features,5J7teKIJGtqVypgYI6sqXj,spotify:track:5J7teKIJGtqVypgYI6sqXj,https://api.spotify.com/v1/tracks/5J7teKIJGtqV...,https://api.spotify.com/v1/audio-analysis/5J7t...,232496,4,5J7teKIJGtqVypgYI6sqXj,Una Vez,Bad Bunny
1,5kIcrM3QVD4BQPFMszQnU1,0.479,0.477,11,-6.886,1,0.0363,0.127,0,0.15,...,audio_features,5kIcrM3QVD4BQPFMszQnU1,spotify:track:5kIcrM3QVD4BQPFMszQnU1,https://api.spotify.com/v1/tracks/5kIcrM3QVD4B...,https://api.spotify.com/v1/audio-analysis/5kIc...,288893,4,5kIcrM3QVD4BQPFMszQnU1,¡Corre!,Jesse & Joy
2,7DTKSwO1kTAJDu9Q5U73UZ,0.716,0.737,7,-5.671,0,0.0851,0.115,0,0.138,...,audio_features,7DTKSwO1kTAJDu9Q5U73UZ,spotify:track:7DTKSwO1kTAJDu9Q5U73UZ,https://api.spotify.com/v1/tracks/7DTKSwO1kTAJ...,https://api.spotify.com/v1/audio-analysis/7DTK...,244107,4,7DTKSwO1kTAJDu9Q5U73UZ,Shining One,BE:FIRST
3,1KYboK62JPwNXnzX9V71G8,0.329,0.866,4,-4.328,1,0.0484,0.0724,0,0.0869,...,audio_features,1KYboK62JPwNXnzX9V71G8,spotify:track:1KYboK62JPwNXnzX9V71G8,https://api.spotify.com/v1/tracks/1KYboK62JPwN...,https://api.spotify.com/v1/audio-analysis/1KYb...,223800,4,1KYboK62JPwNXnzX9V71G8,Stop And Stare,OneRepublic
4,43GfQ1SC17rBWiHkOtdwK8,0.667,0.621,5,-5.697,1,0.0271,0.47,0.000041,0.13,...,audio_features,43GfQ1SC17rBWiHkOtdwK8,spotify:track:43GfQ1SC17rBWiHkOtdwK8,https://api.spotify.com/v1/tracks/43GfQ1SC17rB...,https://api.spotify.com/v1/audio-analysis/43Gf...,192229,4,43GfQ1SC17rBWiHkOtdwK8,Jab Koi Baat - Recreated,Atif Aslam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,7ArrTJ1LwcGAlEO65LUQ7i,0.702,0.623,0,-8.36,1,0.0417,0.322,0,0.0858,...,audio_features,7ArrTJ1LwcGAlEO65LUQ7i,spotify:track:7ArrTJ1LwcGAlEO65LUQ7i,https://api.spotify.com/v1/tracks/7ArrTJ1LwcGA...,https://api.spotify.com/v1/audio-analysis/7Arr...,201198,4,7ArrTJ1LwcGAlEO65LUQ7i,I Don't Wanna Love You Anymore,LANY
496,1koHPgRCutaT5F6C2DjCBw,0.426,0.395,10,-11.121,1,0.0357,0.885,0.0967,0.948,...,audio_features,1koHPgRCutaT5F6C2DjCBw,spotify:track:1koHPgRCutaT5F6C2DjCBw,https://api.spotify.com/v1/tracks/1koHPgRCutaT...,https://api.spotify.com/v1/audio-analysis/1koH...,221987,3,1koHPgRCutaT5F6C2DjCBw,nadie va a pensar en ti mejor que yo - En Vivo,Ed Maverick
497,0QdsKUOus25LYMrqJkOmB8,0.735,0.906,10,-3.658,0,0.11,0.0342,0,0.292,...,audio_features,0QdsKUOus25LYMrqJkOmB8,spotify:track:0QdsKUOus25LYMrqJkOmB8,https://api.spotify.com/v1/tracks/0QdsKUOus25L...,https://api.spotify.com/v1/audio-analysis/0Qds...,190000,4,0QdsKUOus25LYMrqJkOmB8,Nada Ha Cambiao’,Daddy Yankee
498,2PtH42TBhfgAu6jS4RMqEw,0.576,0.703,8,-6.713,1,0.032,0.0701,0.00443,0.116,...,audio_features,2PtH42TBhfgAu6jS4RMqEw,spotify:track:2PtH42TBhfgAu6jS4RMqEw,https://api.spotify.com/v1/tracks/2PtH42TBhfgA...,https://api.spotify.com/v1/audio-analysis/2PtH...,234320,4,2PtH42TBhfgAu6jS4RMqEw,Hoy Te Pierdo,Alfredo Olivas


In [32]:
test_ids = open("spotify_ids.txt", "r")
all_track_ids = test_ids.readlines()
all_track_ids

['7lPN2DXiMsVn7XUKtOW1CS\n',
 '5QO79kh1waicV47BqGRL3g\n',
 '0VjIjW4GlUZAMYd2vXMi3b\n',
 '4MzXwWMhyBbmu6hOcLVD49\n',
 '5Kskr9LcNYa0tpt5f0ZEJx\n',
 '6tDDoYIxWvMLTdKpjFkc1B\n',
 '3VT8hOC5vuDXBsHrR53WFh\n',
 '1xK1Gg9SxG8fy2Ya373oqb\n',
 '6f3Slt0GbA2bPZlz0aIFXN\n',
 '3tjFYV6RSFtuktYl3ZtYcq\n',
 '27OeeYzk6klgBh83TSvGMA\n',
 '2XIc1pqjXV3Cr2BQUGNBck\n',
 '60ynsPSSKe6O3sfwRnIBRf\n',
 '1M4OcYkxAtu3ErzSgDEfoi\n',
 '3YJJjQPAbDT7mGpX3WtQ9A\n',
 '5nujrmhLynf4yMoMtj8AQF\n',
 '1t9WgS8FN0534tLBRwbaxO\n',
 '7vrJn5hDSXRmdXoR30KgF1\n',
 '4saklk6nie3yiGePpBwUoc\n',
 '3FAJ6O0NOHQV8Mc5Ri6ENp\n',
 '0D75ciM842cdUMKSMfAR9y\n',
 '35mvY5S1H3J2QZyna3TFe0\n',
 '6Im9k8u9iIzKMrmV7BWtlF\n',
 '5YYW3yRktprLRr47WK219Y\n',
 '7hxHWCCAIIxFLCzvDgnQHX\n',
 '3VvA1wSxukMLsvXoXtlwWx\n',
 '1diS6nkxMQc3wwC4G1j0bh\n',
 '6ft4hAq6yde8jPZY2i5zLr\n',
 '7qEHsqek33rTcFNT9PFqLf\n',
 '31qCy5ZaophVA81wtlwLc4\n',
 '3iw6V4LH7yPj1ESORX9RIN\n',
 '45bE4HXI0AwGZXfZtMp8JR\n',
 '02MWAaffLxlfxAUY7c5dvx\n',
 '1tkg4EHVoqnhR6iFEXb60y\n',
 '5uEYRdEIh9Bo

In [34]:
all_track_ids = [x.split()[0] for x in all_track_ids]
all_track_ids

['7lPN2DXiMsVn7XUKtOW1CS',
 '5QO79kh1waicV47BqGRL3g',
 '0VjIjW4GlUZAMYd2vXMi3b',
 '4MzXwWMhyBbmu6hOcLVD49',
 '5Kskr9LcNYa0tpt5f0ZEJx',
 '6tDDoYIxWvMLTdKpjFkc1B',
 '3VT8hOC5vuDXBsHrR53WFh',
 '1xK1Gg9SxG8fy2Ya373oqb',
 '6f3Slt0GbA2bPZlz0aIFXN',
 '3tjFYV6RSFtuktYl3ZtYcq',
 '27OeeYzk6klgBh83TSvGMA',
 '2XIc1pqjXV3Cr2BQUGNBck',
 '60ynsPSSKe6O3sfwRnIBRf',
 '1M4OcYkxAtu3ErzSgDEfoi',
 '3YJJjQPAbDT7mGpX3WtQ9A',
 '5nujrmhLynf4yMoMtj8AQF',
 '1t9WgS8FN0534tLBRwbaxO',
 '7vrJn5hDSXRmdXoR30KgF1',
 '4saklk6nie3yiGePpBwUoc',
 '3FAJ6O0NOHQV8Mc5Ri6ENp',
 '0D75ciM842cdUMKSMfAR9y',
 '35mvY5S1H3J2QZyna3TFe0',
 '6Im9k8u9iIzKMrmV7BWtlF',
 '5YYW3yRktprLRr47WK219Y',
 '7hxHWCCAIIxFLCzvDgnQHX',
 '3VvA1wSxukMLsvXoXtlwWx',
 '1diS6nkxMQc3wwC4G1j0bh',
 '6ft4hAq6yde8jPZY2i5zLr',
 '7qEHsqek33rTcFNT9PFqLf',
 '31qCy5ZaophVA81wtlwLc4',
 '3iw6V4LH7yPj1ESORX9RIN',
 '45bE4HXI0AwGZXfZtMp8JR',
 '02MWAaffLxlfxAUY7c5dvx',
 '1tkg4EHVoqnhR6iFEXb60y',
 '5uEYRdEIh9Bo4fpjDd4Na9',
 '6UelLqGlWMcVH1E5c4H7lY',
 '1J14CdDAvBTE1AJYUOwl6C',
 

In [35]:
features = {}
get_features(0, 100)

getting from 0 to 100
getting from 100 to 200
getting from 200 to 300
getting from 300 to 400
getting from 400 to 500
getting from 500 to 600
getting from 600 to 700
getting from 700 to 800
getting from 800 to 900
getting from 900 to 1000
getting from 1000 to 1100
getting from 1100 to 1200


In [36]:
features

{'7lPN2DXiMsVn7XUKtOW1CS': {'danceability': 0.585,
  'energy': 0.436,
  'key': 10,
  'loudness': -8.761,
  'mode': 1,
  'speechiness': 0.0601,
  'acousticness': 0.721,
  'instrumentalness': 1.31e-05,
  'liveness': 0.105,
  'valence': 0.132,
  'tempo': 143.874,
  'type': 'audio_features',
  'id': '7lPN2DXiMsVn7XUKtOW1CS',
  'uri': 'spotify:track:7lPN2DXiMsVn7XUKtOW1CS',
  'track_href': 'https://api.spotify.com/v1/tracks/7lPN2DXiMsVn7XUKtOW1CS',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7lPN2DXiMsVn7XUKtOW1CS',
  'duration_ms': 242014,
  'time_signature': 4},
 '5QO79kh1waicV47BqGRL3g': {'danceability': 0.68,
  'energy': 0.826,
  'key': 0,
  'loudness': -5.487,
  'mode': 1,
  'speechiness': 0.0309,
  'acousticness': 0.0212,
  'instrumentalness': 1.24e-05,
  'liveness': 0.543,
  'valence': 0.644,
  'tempo': 118.051,
  'type': 'audio_features',
  'id': '5QO79kh1waicV47BqGRL3g',
  'uri': 'spotify:track:5QO79kh1waicV47BqGRL3g',
  'track_href': 'https://api.spotify.com/v1/tr

In [37]:
tracks = pd.DataFrame.from_dict(features)
tracks = tracks.T
tracks = tracks.reset_index().rename(columns={'index' : 'song_id'})
tracks

,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,7lPN2DXiMsVn7XUKtOW1CS,0.585,0.436,10,-8.761,1,0.0601,0.721,0.000013,0.105,0.132,143.874,audio_features,7lPN2DXiMsVn7XUKtOW1CS,spotify:track:7lPN2DXiMsVn7XUKtOW1CS,https://api.spotify.com/v1/tracks/7lPN2DXiMsVn...,https://api.spotify.com/v1/audio-analysis/7lPN...,242014,4
1,5QO79kh1waicV47BqGRL3g,0.68,0.826,0,-5.487,1,0.0309,0.0212,0.000012,0.543,0.644,118.051,audio_features,5QO79kh1waicV47BqGRL3g,spotify:track:5QO79kh1waicV47BqGRL3g,https://api.spotify.com/v1/tracks/5QO79kh1waic...,https://api.spotify.com/v1/audio-analysis/5QO7...,215627,4
2,0VjIjW4GlUZAMYd2vXMi3b,0.514,0.73,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,audio_features,0VjIjW4GlUZAMYd2vXMi3b,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4
3,4MzXwWMhyBbmu6hOcLVD49,0.731,0.573,4,-10.059,0,0.0544,0.401,0.000052,0.113,0.145,109.928,audio_features,4MzXwWMhyBbmu6hOcLVD49,spotify:track:4MzXwWMhyBbmu6hOcLVD49,https://api.spotify.com/v1/tracks/4MzXwWMhyBbm...,https://api.spotify.com/v1/audio-analysis/4MzX...,205090,4
4,5Kskr9LcNYa0tpt5f0ZEJx,0.907,0.393,4,-7.636,0,0.0539,0.451,0.000001,0.135,0.202,104.949,audio_features,5Kskr9LcNYa0tpt5f0ZEJx,spotify:track:5Kskr9LcNYa0tpt5f0ZEJx,https://api.spotify.com/v1/tracks/5Kskr9LcNYa0...,https://api.spotify.com/v1/audio-analysis/5Ksk...,205458,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157,4lUmnwRybYH7mMzf16xB0y,0.596,0.65,9,-5.167,1,0.337,0.138,0,0.14,0.188,133.997,audio_features,4lUmnwRybYH7mMzf16xB0y,spotify:track:4lUmnwRybYH7mMzf16xB0y,https://api.spotify.com/v1/tracks/4lUmnwRybYH7...,https://api.spotify.com/v1/audio-analysis/4lUm...,257428,4
1158,1fzf9Aad4y1RWrmwosAK5y,0.588,0.85,4,-6.431,1,0.0318,0.168,0.00202,0.0465,0.768,93.003,audio_features,1fzf9Aad4y1RWrmwosAK5y,spotify:track:1fzf9Aad4y1RWrmwosAK5y,https://api.spotify.com/v1/tracks/1fzf9Aad4y1R...,https://api.spotify.com/v1/audio-analysis/1fzf...,187310,4
1159,3E3pb3qH11iny6TFDJvsg5,0.754,0.66,0,-6.811,1,0.267,0.179,0,0.194,0.316,83.0,audio_features,3E3pb3qH11iny6TFDJvsg5,spotify:track:3E3pb3qH11iny6TFDJvsg5,https://api.spotify.com/v1/tracks/3E3pb3qH11in...,https://api.spotify.com/v1/audio-analysis/3E3p...,209299,4
1160,3yTkoTuiKRGL2VAlQd7xsC,0.584,0.836,0,-4.925,1,0.079,0.0558,0,0.0663,0.484,104.973,audio_features,3yTkoTuiKRGL2VAlQd7xsC,spotify:track:3yTkoTuiKRGL2VAlQd7xsC,https://api.spotify.com/v1/tracks/3yTkoTuiKRGL...,https://api.spotify.com/v1/audio-analysis/3yTk...,202204,4


In [ ]:
tracks.to_csv("test_tracks.csv")